## Тестирование производительности OLAP-хранилищ Clickhouse и Vertica

В рамках исследования были рассмотрены Clickhouse и Vertica.

In [ ]:
!docker-compose up -d

#### Загрузка тестовых данных

Структура тестовой таблицы

| ----------- | ----------- | | id | UUID | | user_id | UUID | | movie_id | UUID | | viewed_frame | Int |

In [ ]:
!export PYTHONPATH="${PYTHONPATH}:${PWD}/../.."